In [1]:
from glob import glob
import os
import numpy as np
import matplotlib as plt
from torchvision import transforms
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import torchvision
import torch.nn.functional as F

# 라이브러리 불러오기

In [2]:
path = './dataset/'
files = glob(os.path.join(path, '*/*/*.png'))

print ( "Total no of images", len(files) )

# 현재 데이터셋의 수를 받음

Total no of images 168


In [3]:
simple_transform = transforms.Compose([transforms.Resize((60,60))
                        , transforms.ToTensor()
                        , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train = ImageFolder('dataset/train/', simple_transform)
valid = ImageFolder('dataset/valid/', simple_transform)

print(train.class_to_idx)
print(train.classes)

# 라벨링 체크 및 데이터 로더 1

{'In': 0, 'Out': 1}
['In', 'Out']


In [4]:
train_data_gen = torch.utils.data.DataLoader(train,shuffle=True,batch_size=4,num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=4,num_workers=3)

# 데이터 로더 2

In [5]:
dataset_sizes = {'train':len(train_data_gen.dataset),'valid':len(valid_data_gen.dataset)}

dataloaders = {'train':train_data_gen,'valid':valid_data_gen}

# 데이터 불러오기

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64, eps=1e-3),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64, eps=1e-3),
            nn.ReLU(),
        )

        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128, eps=1e-3),
            nn.ReLU(),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128, eps=1e-3),
            nn.ReLU(),
        )

        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256, eps=1e-3),
            nn.ReLU(),
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256, eps=1e-3),
            nn.ReLU(),
        )

        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv7 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512, eps=1e-3),
            nn.ReLU(),
        )
        
        self.conv8 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512, eps=1e-3),
            nn.ReLU(),
        )
        
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512, eps=1e-3),
            nn.ReLU(),
        )
        self.conv10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512, eps=1e-3),
            nn.ReLU(),
        )

        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.linear1 = nn.Sequential(
            nn.Linear(4608, 512),
            nn.ReLU(),
        )
        self.linear2 = nn.Linear(512, 512)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.maxpool3(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.maxpool4(x)

        x = torch.flatten(x, 1)

        x = self.linear1(x)
        x = self.linear2(x)

        return x

In [7]:
model     = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
model

# 모델 선언

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ce

In [9]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [11]:
EPOCHS     = 20
BATCH_SIZE = 64

DEVICE = torch.device("cpu")

In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_data_gen, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, valid_data_gen)
    train_loss, train_accuracy = evaluate(model, train_data_gen)
    
    print('[{}] Train Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, train_loss, train_accuracy))
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))
    
print("""

학습이 완료되었습니다. """)

Train Epoch: 1 [0/138 (0%)]	Loss: 6.576874
Train Epoch: 1 [80/138 (57%)]	Loss: 0.859234
[1] Train Loss: 1.1087, Accuracy: 52.17%
[1] Test Loss: 0.9416, Accuracy: 50.00%
Train Epoch: 2 [0/138 (0%)]	Loss: 0.874874
Train Epoch: 2 [80/138 (57%)]	Loss: 1.416640
[2] Train Loss: 1.5457, Accuracy: 70.29%
[2] Test Loss: 1.4899, Accuracy: 70.00%
Train Epoch: 3 [0/138 (0%)]	Loss: 2.094484
Train Epoch: 3 [80/138 (57%)]	Loss: 0.508054
[3] Train Loss: 0.1667, Accuracy: 93.48%
[3] Test Loss: 0.8434, Accuracy: 80.00%
Train Epoch: 4 [0/138 (0%)]	Loss: 0.010135
Train Epoch: 4 [80/138 (57%)]	Loss: 0.097742
[4] Train Loss: 0.8363, Accuracy: 81.16%
[4] Test Loss: 0.6863, Accuracy: 80.00%
Train Epoch: 5 [0/138 (0%)]	Loss: 0.471290
Train Epoch: 5 [80/138 (57%)]	Loss: 0.730839
[5] Train Loss: 0.0689, Accuracy: 97.10%
[5] Test Loss: 0.6414, Accuracy: 80.00%
Train Epoch: 6 [0/138 (0%)]	Loss: 0.008617
Train Epoch: 6 [80/138 (57%)]	Loss: 0.064200
[6] Train Loss: 0.0527, Accuracy: 97.83%
[6] Test Loss: 0.6419, Acc